In [1]:
from IPython.core.display import display, HTML,display_html
from IPython.display import display, Markdown, clear_output
from ipywidgets import interact, interact_manual
import ipywidgets as widgets
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import seaborn as sns
%load_ext autoreload
%autoreload 2

Loading Libs

In [2]:
df = pd.read_csv('./qbinsModel.csv')

In [5]:
qbins =20
out, bins = pd.qcut(df['riskGood'],qbins,duplicates='drop', retbins=True)

### Create Groups

In [6]:
def get_bins(x,binz):
    for i in range(len(binz)):
        if i == 0:
            if x <= binz[i]:
                return i
        elif i == len(binz)-1:
            if x >= binz[i]:
                return i+1
            elif x >= binz[i-1] and x < binz[i]:
                return i
        else:
            if x >= binz[i-1] and x < binz[i]:
                return i

In [7]:
def getRating(x):
    rating = ['A1','A2','A3','B1','B2','B3','C1','C2','C3','D1','D2','D3','E','F']
    thresh = [0.03,0.06,0.08,0.1,0.15,0.2,0.25,0.35,0.5,0.6,0.7,0.8,0.9]
    for i in range(len(thresh)):
        if i == 0:
            if x <= thresh[i]:
                return rating[i]
        elif i == len(thresh)-1:
            if x >= thresh[i]:
                return rating[i+1]
            elif x >= thresh[i-1] and x < thresh[i]:
                return rating[i]
        else:
            if x >= thresh[i-1] and x < thresh[i]:
                return rating[i]

In [46]:
def createRiskGroup(df,binz):
    groups = []
    for i in range(1,len(binz)+1):
        try:
            groups.append({
                'group' : i, 'threshold' : binz[i-1] , 
                'bad(%)' : df['bad'][df['risk_group'] == i].value_counts(normalize=True).sort_index().values[1]*100 , 
                'pop(%)' : (df[df['risk_group'] == i].shape[0]/df.shape[0])*100 , 
                'rating' : getRating(df['bad'][df['risk_group'] == i].value_counts(normalize=True).sort_index().values[1]) 
            })
        except:
            groups.append({
                'group' : i, 'threshold' : binz[i-1] , 
                'bad(%)' : 0, 
                'pop(%)' : (df[df['risk_group'] == i].shape[0]/df.shape[0])*100, 
                'rating' : getRating(0) 
            })
    return groups

In [13]:
df['risk_group'] = df['riskGood'].map(lambda x: get_bins(x,bins[1:-1]))
grouped = pd.DataFrame(createRiskGroup(df,bins[1:-1]))

In [14]:
grouped

,group,threshold,bad(%),pop(%),rating
0,1,0.765172,32.820513,4.994877,C2
1,2,0.782653,27.692308,4.994877,C2
2,3,0.789908,24.102564,4.994877,C1
3,4,0.798604,26.153846,4.994877,C2
4,5,0.804971,20.512821,4.994877,C1
5,6,0.808662,18.877551,5.020492,B3
6,7,0.810714,18.461538,4.994877,B3
7,8,0.814855,18.461538,4.994877,B3
8,9,0.819344,14.871795,4.994877,B2
9,10,0.824133,17.948718,4.994877,B3


### Decision Process

In [47]:
df['risk_group'] = df['riskGood'].map(lambda x: get_bins(x,bins[1:-1]))
grouped = pd.DataFrame(createRiskGroup(df,bins[1:-1]))

changes = 1
while changes != 0:
    newBins = list(grouped['threshold'].values)
    changes = 0
    i = 0
    # kinda recursive
    while i <= grouped.shape[0]-2:
        
        if grouped.loc[i+1,'bad(%)'] > 35:
            
            if grouped.loc[i,'rating'] == grouped.loc[i+1,'rating']:
                
                newBins.remove(grouped.loc[i,'threshold'])
                i = i + 1
                changes = changes + 1
            
            elif np.abs(grouped.loc[i,'bad(%)'] - grouped.loc[i+1,'bad(%)']) < 15 and (grouped.loc[i,'pop(%)'] + grouped.loc[i+1,'pop(%)']) < 15 and grouped.loc[i+1,'bad(%)'] > 2:
                
                #print('3 - Dropping',str(grouped.loc[i,'threshold']),'Leaving',str(grouped.loc[i+1,'threshold']))
                
                newBins.remove(grouped.loc[i,'threshold'])
                i = i + 1
                changes = changes + 1
                
            else:
                pass
                
        else:
            
            if grouped.loc[i,'rating'] == grouped.loc[i+1,'rating']:
                
                newBins.remove(grouped.loc[i,'threshold'])
                i = i + 1
                changes = changes + 1
            
            elif (grouped.loc[i,'pop(%)']) < 2 and np.abs(grouped.loc[i,'bad(%)'] - grouped.loc[i+1,'bad(%)']) < grouped.loc[i+1,'bad(%)']/10 and grouped.loc[i+1,'bad(%)'] < 2:
                
                print('0 - Dropping',str(grouped.loc[i,'threshold']),'Leaving',str(grouped.loc[i+1,'threshold']))
                
                newBins.remove(grouped.loc[i,'threshold'])
                i = i + 1
                changes = changes + 1
                
            elif np.abs(grouped.loc[i,'bad(%)'] - grouped.loc[i+1,'bad(%)']) < grouped.loc[i+1,'bad(%)']/10 and (grouped.loc[i,'pop(%)'] + grouped.loc[i+1,'pop(%)']) < 10 and  grouped.loc[i+1,'bad(%)'] > 2:
                
                print('1 - Dropping',str(grouped.loc[i,'threshold']),'Leaving',str(grouped.loc[i+1,'threshold']))
                
                newBins.remove(grouped.loc[i,'threshold'])
                i = i + 1
                changes = changes + 1
                
            elif grouped.loc[i,'bad(%)'] < grouped.loc[i+1,'bad(%)']:
                
                print('1 - Dropping',str(grouped.loc[i,'threshold']),'Leaving',str(grouped.loc[i+1,'threshold']))
                
                newBins.remove(grouped.loc[i,'threshold'])
                i = i + 1
                changes = changes + 1
                
            else:
                pass
        
        if i <= grouped.shape[0]:
            i = i + 1
        
    newBins = sorted(list(set(newBins+[0])))
    df['risk_group'] = df['riskGood'].map(lambda x: get_bins(x,newBins))
    grouped = pd.DataFrame(createRiskGroup(df,newBins))
    #display(grouped)
grouped

1 - Dropping 0.7899081413435058 Leaving 0.7986036475517472
1 - Dropping 0.8193437580095042 Leaving 0.8241334881697593
1 - Dropping 0.8281373806183987 Leaving 0.8337444777635837
1 - Dropping 0.8503171456448666 Leaving 0.8582895118582936


,group,threshold,bad(%),pop(%),rating
0,1,0.000000,22.807018,59.861680,C1
1,2,0.833744,15.151515,10.143443,B3
2,3,0.843915,9.487179,9.989754,B1
3,4,0.858290,6.445993,14.702869,A3
4,5,0.890819,3.381643,5.302254,A2


In [50]:
def printThresholds(binz):
    print('def get_bins(x):')
    for i in range(len(binz)):
        if i == 0:
            print('\tif x <=',np.round(binz['threshold'][i+1],5),':')
            print('\t\treturn',i+1,',"'+binz['rating'][i]+'"')
        elif i == len(binz)-1:
            print('\telif x > ',np.round(binz['threshold'][i],5),':')
            print('\t\treturn',i+1,',"'+binz['rating'][i]+'"')
        else:
            print('\telif x > ',np.round(binz['threshold'][i],5),'and  x <= ',np.round(binz['threshold'][i+1],4),':')
            print('\t\treturn',i+1,',"'+binz['rating'][i]+'"')

In [51]:
printThresholds(grouped[['threshold','rating']].sort_values('threshold'))

def get_bins(x):
	if x <= 0.83374 :
		return 1 ,"C1"
	elif x >  0.83374 and  x <=  0.8439 :
		return 2 ,"B3"
	elif x >  0.84392 and  x <=  0.8583 :
		return 3 ,"B1"
	elif x >  0.85829 and  x <=  0.8908 :
		return 4 ,"A3"
	elif x >  0.89082 :
		return 5 ,"A2"


FIM